## Alpaca to Odia Translation

#Setup

In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

**IMPORTANT : Restart your run-time first and then run the cells below.**

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor

BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [ ]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "eng_Latn", "ory_Orya"

In [ ]:
!pip install datasets sentencepiece huggingface_hub -q

In [ ]:
import os
import torch
import json
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from huggingface_hub import login

# Download and prepare the Alpaca dataset

In [ ]:
print("Loading Alpaca dataset...")
!wget -q https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

with open('alpaca_data.json', 'r') as f:
    alpaca_data = json.load(f)

print(f"Loaded {len(alpaca_data)} examples from Alpaca dataset")

# Process the Alpaca dataset

In [ ]:
print("Processing Alpaca dataset for translation...")
instructions = [item["instruction"] for item in alpaca_data]
inputs = [item["input"] if item["input"] else "" for item in alpaca_data]
outputs = [item["output"] for item in alpaca_data]

# Translating in parts
print("Translating instructions...")
odia_instructions = batch_translate(instructions, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)


print("Translating inputs...")
odia_inputs = batch_translate(inputs, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)


print("Translating outputs...")
odia_outputs = batch_translate(outputs, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)



# Create Odia Alpaca dataset

In [ ]:
odia_alpaca_data = []
for i in range(len(alpaca_data)):
    odia_alpaca_data.append({
        "instruction": odia_instructions[i],
        "input": odia_inputs[i],
        "output": odia_outputs[i],
        "original_instruction": instructions[i],
        "original_input": inputs[i],
        "original_output": outputs[i]
    })

# Save the dataset
with open('alpaca_odia_data.json', 'w', encoding='utf-8') as f:
    json.dump(odia_alpaca_data, f, ensure_ascii=False, indent=4)


In [ ]:

print("Sample for verification...")
# Few samples for manual verification
samples = pd.DataFrame(odia_alpaca_data[:10])
samples.head()

# Create HuggingFace Dataset and push to Hub

In [ ]:
print("Preparing dataset for HuggingFace...")
from datasets import Dataset

dataset = Dataset.from_pandas(pd.DataFrame(odia_alpaca_data))

# Save dataset to Hub
dataset_name = "sumankumarbhadra/alpaca-odia"
dataset.push_to_hub(
    dataset_name,
    private=False,
    token=hf_token
)

print(f"Dataset uploaded to HuggingFace Hub as '{dataset_name}'")